In [249]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [250]:
from utils import top_40_cols

df = pd.read_csv('../Dataset/train.csv/train.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month

df.drop(['timestamp'], axis=1, inplace=True)
target = df['price_doc']

columns_to_drop = [column for column in df.columns if column not in top_40_cols]
df.drop(columns=columns_to_drop, axis=1, inplace=True)
missing_column = [column for column in top_40_cols if column not in df.columns]
print(missing_column)
df['price_doc'] = target

df

['product_type_Investment']


,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,metro_min_avto,...,cafe_count_3000_price_1500,cafe_count_3000_price_2500,sport_count_3000,office_sqm_5000,cafe_count_5000_na_price,cafe_count_5000_price_2500,cafe_count_5000_price_high,year,month,price_doc
0,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,Bibirevo,2.590241,...,16,3,21,807385,12,9,0,2011,8,5850000
1,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,Nagatinskij Zaton,0.936700,...,4,2,19,2690465,9,15,0,2011,8,6000000
2,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,Tekstil'shhiki,2.120999,...,9,3,20,1478160,10,10,0,2011,8,5700000
3,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,Mitino,1.489049,...,10,3,18,244166,4,11,1,2011,9,13100000
4,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,Basmannoe,1.257186,...,262,149,77,8404624,143,319,17,2011,9,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,44,27.0,7.0,9.0,1975.0,2.0,6.0,3.0,Otradnoe,1.384021,...,15,5,29,838601,18,15,0,2015,6,7400000
30467,86,59.0,3.0,9.0,1935.0,4.0,10.0,3.0,Tverskoe,1.060577,...,230,155,80,9949843,136,313,24,2015,6,25000000
30468,45,NaN,10.0,20.0,NaN,1.0,1.0,1.0,Poselenie Vnukovskoe,2.152792,...,2,1,6,117300,1,1,0,2015,6,6970959
30469,64,32.0,5.0,15.0,2003.0,2.0,11.0,2.0,Obruchevskoe,3.377814,...,26,13,33,1225712,11,22,1,2015,6,13500000


In [251]:
print("Columns with NAN: ", df.columns[df.isna().any()].tolist())
# columns with NAN are ['life_sq', 'floor', 'max_floor', 'build_year', 'num_room', 'kitch_sq', 'state', 'metro_min_walk']
# first step is to fill in NaN columns with some values.
# we take different approaches for different columns

# For life_sq, I think it is acceptable that we replace NaN values with the full_sq values of those rows
df['life_sq'].fillna(df['full_sq'], inplace=True)
print("Columns with NAN: ", df.columns[df.isna().any()].tolist())



Columns with NAN:  ['life_sq', 'floor', 'max_floor', 'build_year', 'num_room', 'kitch_sq', 'state', 'metro_min_walk']
Columns with NAN:  ['floor', 'max_floor', 'build_year', 'num_room', 'kitch_sq', 'state', 'metro_min_walk']


In [252]:
# for max_floor, we could fill NaN with the median max_floor of properties in the same sub_area
sub_area_medians = df.groupby('sub_area')['max_floor'].median().reset_index()
# sub_area_medians['max_floor'] = np.ceil(sub_area_medians['max_floor'])
df = df.merge(sub_area_medians, on='sub_area', suffixes=('', '_median'), how='left')
df['max_floor'].fillna(df['max_floor_median'], inplace=True)
df.drop(columns='max_floor_median', inplace=True)

# and then for floor, we just fill NaN with the max_floor
df['floor'].fillna(df['max_floor'], inplace=True)

# finally we replace the max_floor with the floor, if there are any value of floor greater than max_floor(e.g row 63)
df['max_floor'] = df.apply(lambda row: row['floor'] if row['floor'] > row['max_floor'] else row['max_floor'], axis=1)

df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,metro_min_avto,...,cafe_count_3000_price_1500,cafe_count_3000_price_2500,sport_count_3000,office_sqm_5000,cafe_count_5000_na_price,cafe_count_5000_price_2500,cafe_count_5000_price_high,year,month,price_doc
0,43,27.0,4.0,12.0,NaN,NaN,NaN,NaN,Bibirevo,2.590241,...,16,3,21,807385,12,9,0,2011,8,5850000
1,34,19.0,3.0,17.0,NaN,NaN,NaN,NaN,Nagatinskij Zaton,0.936700,...,4,2,19,2690465,9,15,0,2011,8,6000000
2,43,29.0,2.0,12.0,NaN,NaN,NaN,NaN,Tekstil'shhiki,2.120999,...,9,3,20,1478160,10,10,0,2011,8,5700000
3,89,50.0,9.0,17.0,NaN,NaN,NaN,NaN,Mitino,1.489049,...,10,3,18,244166,4,11,1,2011,9,13100000
4,77,77.0,4.0,9.0,NaN,NaN,NaN,NaN,Basmannoe,1.257186,...,262,149,77,8404624,143,319,17,2011,9,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,44,27.0,7.0,9.0,1975.0,2.0,6.0,3.0,Otradnoe,1.384021,...,15,5,29,838601,18,15,0,2015,6,7400000
30467,86,59.0,3.0,9.0,1935.0,4.0,10.0,3.0,Tverskoe,1.060577,...,230,155,80,9949843,136,313,24,2015,6,25000000
30468,45,45.0,10.0,20.0,NaN,1.0,1.0,1.0,Poselenie Vnukovskoe,2.152792,...,2,1,6,117300,1,1,0,2015,6,6970959
30469,64,32.0,5.0,15.0,2003.0,2.0,11.0,2.0,Obruchevskoe,3.377814,...,26,13,33,1225712,11,22,1,2015,6,13500000


In [253]:
# we do the same for build_year, fill NaN with the median build_year of properties in the same sub_area
sub_area_medians = df.groupby('sub_area')['build_year'].median().reset_index()
# sub_area_medians['build_year'] = np.ceil(sub_area_medians['build_year'])
df = df.merge(sub_area_medians, on='sub_area', suffixes=('', '_median'), how='left')
df['build_year'].fillna(df['build_year_median'], inplace=True)
df.drop(columns='build_year_median', inplace=True)

In [254]:
# for num_room, we shall split the data into different ranges of full_sq value and calculate the average num_room for each range
# then, we replace the num_room NaN values depending on which range the row's full_sq belongs to
# Define the ranges for 'full_sq' bins
bins = [0, 30, 52, 80, float('inf')]  # these values are eyeballed
print(bins)

# Use pd.cut to create bins for 'full_sq'
df['full_sq_bins'] = pd.cut(df['full_sq'], bins=bins)

# Calculate the average 'num_room' for each 'full_sq' range
# num_room_averages = df.groupby('full_sq_bins')['num_room'].transform('mean')
num_room_averages = df.groupby('full_sq_bins')['num_room'].transform(lambda x: np.ceil(x.mean()))
df['num_room'].fillna(num_room_averages, inplace=True)
df.drop(columns='full_sq_bins', inplace=True)

[0, 30, 52, 80, inf]


C:\Users\Yu_Hao\AppData\Local\Temp\ipykernel_18516\1430556226.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  num_room_averages = df.groupby('full_sq_bins')['num_room'].transform(lambda x: np.ceil(x.mean()))


In [255]:
# for kitch_sq, we shall split the data into different ranges of life_sq value and calculate the average kitch_sq for each range
# then, we replace the kitch_sq NaN values depending on which range the row's life_sq belongs to
num_bins = 4
df['life_sq_bins'] = pd.cut(df['life_sq'], bins=num_bins)

In [256]:
pd.set_option('display.max_rows', None)
display(df[df['sub_area'] == 'Novogireevo'])
pd.reset_option('display.max_rows')

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,sub_area,metro_min_avto,...,cafe_count_3000_price_2500,sport_count_3000,office_sqm_5000,cafe_count_5000_na_price,cafe_count_5000_price_2500,cafe_count_5000_price_high,year,month,price_doc,life_sq_bins
55,43,25.0,1.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,1.571251,...,2,29,384344,3,7,0,2011,10,5500000,"(-7.478, 1869.5]"
63,39,39.0,10.0,10.0,1968.0,2.0,NaN,NaN,Novogireevo,1.687127,...,2,19,337974,3,4,0,2011,10,2080000,"(-7.478, 1869.5]"
294,33,20.0,9.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,1.037491,...,2,21,337974,3,4,0,2011,11,1000000,"(-7.478, 1869.5]"
462,35,19.0,4.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,1.598338,...,2,34,726999,6,8,0,2011,11,4950000,"(-7.478, 1869.5]"
567,77,45.0,11.0,11.0,1968.0,3.0,NaN,NaN,Novogireevo,1.014522,...,2,26,351244,3,5,0,2011,12,9100000,"(-7.478, 1869.5]"
616,35,19.0,10.0,10.0,1968.0,2.0,NaN,NaN,Novogireevo,0.989985,...,2,28,384344,4,6,0,2011,12,5800000,"(-7.478, 1869.5]"
681,44,28.0,1.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,2.214110,...,2,31,384344,3,7,0,2011,12,5150000,"(-7.478, 1869.5]"
741,43,28.0,2.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,1.669941,...,2,28,380244,4,7,0,2011,12,5250000,"(-7.478, 1869.5]"
786,114,70.0,13.0,13.0,1968.0,4.0,NaN,NaN,Novogireevo,1.328840,...,2,34,713099,6,8,0,2012,1,23500000,"(-7.478, 1869.5]"
790,43,28.0,2.0,9.0,1968.0,2.0,NaN,NaN,Novogireevo,1.254381,...,2,35,693624,6,7,0,2012,1,6200000,"(-7.478, 1869.5]"
